# Reconocimiento de audio y conversión a texto con Whisper.

## Importación de Bibliotecas

Los siguientes comandos instalarán los paquetes de Python necesarios para grabar fragmentos de audio y utilizar los modelos Whisper para la transcripción de voz a texto.

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

También ocuparemos lo siguiente para poder grabar audio desde este cuaderno y poder procesar los archivos resultantes.

In [ ]:
!apt install ffmpeg
!apt-get install libportaudio2

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Realizamos una grabación de prueba

Activaremos algunos widgets que nos proporciona Colab para poder realizar la grabación.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Pulsa el botón circular y empieza a hablar. Puede que no lo parezca, pero el widget estará capturando sonido. Pulsa de nuevo el botón del círculo cuando hayas terminado. El widget empezará inmediatamente a reproducir lo que ha capturado.

Puedes usar la siguiente frase de Albert Einstein si no se te ocurre algo que grabar: "Una persona que nunca ha cometido un error nunca intenta nada nuevo"

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

El formato de audio capturado anteriormente no es legible por PyTorch. En este paso, convertimos nuestra grabación a un formato que PyTorch pueda entender.

In [ ]:
with open('grabacion.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i grabacion.webm -ac 1 -f wav mi_grabacion.wav -y -hide_banner -loglevel panic

## Opciones de selección

Whisper es capaz de realizar transcripciones para muchos idiomas (aunque funciona mejor para algunos idiomas que para otros).

Whisper también es capaz de detectar el idioma de entrada. Sin embargo, para estar seguros, podemos decirle explícitamente a Whisper qué idioma debe esperar.

In [ ]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE
language_list = list(language_options.keys())

In [ ]:
lang_dropdown = widgets.Dropdown(options=language_list, value='spanish')
output = widgets.Output()
display(lang_dropdown)

Whisper también es capaz de realizar varias tareas, como la transcripción sólo en inglés, la traducción de cualquier idioma al inglés y la transcripción de en idioma original.

A continuación puede seleccionar «transcripción» (que producirá texto en el mismo idioma que el de entrada) o «traducción» (que transcribirá de un idioma diferente al inglés a inglés).

![Capacidades de Whisper](https://cdn.openai.com/whisper/draft-20220920a/asr-training-data-desktop.svg)

Imagen tomada de [Introducción a Whisper](https://openai.com/blog/whisper/) by OpenAI

In [ ]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)

## Cargamos el modelo Whisper

Whisper viene en cinco tamaños de modelo, cuatro de los cuales también tienen una versión optimizada sólo en inglés. Este cuaderno carga modelos de tamaño «base» (más grandes que «tiny» pero más pequeños que los demás), que requieren aproximadamente 1 GB de RAM.

Si ha seleccionado inglés arriba, la celda de abajo cargará la versión optimizada sólo en inglés. En caso contrario, cargará el modelo multilingüe.


In [ ]:
if lang_dropdown.value == "english":
  model = whisper.load_model("base.en")
else:
  model = whisper.load_model("base")
print(
    f"El modelo es {'multilingual' if model.is_multilingual else 'English-only'} "
    f"y tiene {sum(np.prod(p.shape) for p in model.parameters()):,} parámetros."
)

Por último, establezcamos el resto de nuestras opciones de tarea e idioma a continuación y veamos lo que tenemos. Comprueba que las opciones de tarea e idioma son correctas, pero no te preocupes por el resto de valores predeterminados.

In [ ]:
options = whisper.DecodingOptions(language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=True)
options

## Poniendo Whisper a prueba

Todo lo que queda por hacer ahora es introducir nuestro audio en Whisper.
La celda de abajo realiza los últimos pasos de procesamiento para que esto suceda. Primero, carga nuestro archivo de audio listo para PyTorch. A continuación, divide el audio en segmentos de 30 segundos. Crea un espectrograma log-mel del audio y lo introduce en Whisper junto con las opciones que establecimos anteriormente.


In [ ]:
audio = whisper.load_audio("mi_grabacion.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
result = model.decode(mel, options)

In [ ]:
result.text